## Get OpenSky API Data

In [1]:
%pip install azure.eventhub

StatementMeta(, c643bb30-3e27-4f5e-a7e0-0eddac306ad6, -1, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 3.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /nfs4/pyenv-604339b1-bd01-4a18-b849-e99ff1ce409b/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import asyncio
import requests
import json

from azure.eventhub import EventData
from azure.eventhub.aio import EventHubProducerClient

EVENT_HUB_CONNECTION_STR = "Endpoint=sb://eventsforfabric.servicebus.windows.net/;SharedAccessKeyName=manage;SharedAccessKey=wr/fsIbvRfljovPzpAYLvPf/XI3JZLAtz+AEhMnlKIw=;EntityPath=opensky"
EVENT_HUB_NAME = "opensky"

async def main_coroutine():
    print("Starting...")

    while True:
        await asyncio.sleep(1)

        producer = EventHubProducerClient.from_connection_string(conn_str=EVENT_HUB_CONNECTION_STR, eventhub_name=EVENT_HUB_NAME)
        
        url = "https://opensky-network.org/api/states/all?lamin=24.396308&lomin=-124.566009&lamax=49.384358&lomax=-66.934570"
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        data = response.json()

        states = data.get('states', [])
        
        async with producer:
            for state in states:
                try:
                    event_data_batch = await producer.create_batch()

                    event_data_batch.add(EventData(json.dumps(state)))

                    # Send the batch of events to the event hub.
                    await producer.send_batch(event_data_batch)
                    print(f"Sent to Azure successfully: {state}")

                except ValueError as ve:
                    print(f"Value Error: {ve}")

# In a Jupyter notebook, you can directly await a coroutine without needing to manage the event loop
await main_coroutine()

StatementMeta(, c643bb30-3e27-4f5e-a7e0-0eddac306ad6, 9, Submitted, Running)

Starting...
Sent to Azure successfully: ['ab1644', 'UAL259  ', 'United States', 1693603910, 1693603910, -111.087, 36.2138, 11879.58, False, 258.97, 58.24, -0.33, None, 12550.14, '3110', False, 0]
Sent to Azure successfully: ['a3fef5', 'N357BP  ', 'United States', 1693603909, 1693603909, -104.763, 39.0001, 2141.22, False, 43.25, 116.87, 0, None, 2209.8, None, False, 0]
Sent to Azure successfully: ['a09281', 'N136LM  ', 'United States', 1693603908, 1693603909, -84.3293, 33.8602, 396.24, False, 28.76, 116.57, -2.6, None, 480.06, None, False, 0]
Sent to Azure successfully: ['ad5612', 'N95897  ', 'United States', 1693603909, 1693603909, -122.5852, 47.8403, 1432.56, False, 65.88, 88.21, 0.33, None, 1409.7, None, False, 0]
Sent to Azure successfully: ['aa8c39', 'UAL1828 ', 'United States', 1693603909, 1693603909, -81.5142, 34.0039, 11277.6, False, 258.73, 349.81, 0, None, 12009.12, '5542', False, 0]
Sent to Azure successfully: ['a46e70', 'LXJ385  ', 'United States', 1693603909, 1693603909, -8